In [2]:
import scipy
import numpy
import scipy.io
import matplotlib.pyplot as plt
import pyamg
from scipy.sparse.linalg import cg

In [11]:
A=scipy.io.mmread("/home/yiwei/results_new/test_refine_0.00/2d_bar_lowE/AMGCL/square_beam2d_structed/bar_2d/P1/ref0/block2/Thread32/0/output/stiffness.mat")
A_csr=A.tocsr()

In [4]:
b = scipy.io.mmread("/home/yiwei/results_new/test_refine_0.00/2d_bar_lowE/AMGCL/square_beam2d_structed/bar_2d/P1/ref0/block2/Thread32/0/output/new_matrix/rhs.mtx")

In [12]:
ml = pyamg.smoothed_aggregation_solver(A_csr, max_levels=10, max_coarse=5, keep=True,symmetry='symmetric',strength=[('symmetric', {'theta':0.00})])
print(ml)

multilevel_solver
Number of Levels:     3
Operator Complexity:  1.053
Grid Complexity:      1.083
Coarse Solver:        'pinv2'
  level   unknowns     nonzeros
    0          132         1262 [94.96%]
    1           10           66 [ 4.97%]
    2            1            1 [ 0.08%]



In [6]:
# ml_ad = pyamg.aggregation.adaptive_sa_solver(A_csr,pdef=True,num_candidates=6, max_levels=10, max_coarse=500, keep=True,symmetry='symmetric',strength=[('symmetric', {'theta':0.08})])
# print(ml_ad)

/home/yiwei/anaconda3/lib/python3.9/site-packages/pyamg/aggregation/adaptive.py:250: RuntimeWarning: divide by zero encountered in double_scalars
  B = (1.0/norm(B, 'inf')) * B
/home/yiwei/anaconda3/lib/python3.9/site-packages/pyamg/aggregation/adaptive.py:250: RuntimeWarning: invalid value encountered in multiply
  B = (1.0/norm(B, 'inf')) * B


ValueError: array must not contain infs or NaNs

In [7]:
def solve_sparse(A, b,M):
    num_iters = 0

    def callback(xk):
        nonlocal num_iters
        num_iters+=1

    x,status=scipy.sparse.linalg.cg(A, b,tol=1e-8, callback=callback,M=M)
    return x,status,num_iters

In [8]:
M = ml.aspreconditioner(cycle='V')  
M_ad = ml.aspreconditioner(cycle='V') # preconditioner
x, info,iter = solve_sparse(A, b, M=M)
x_ad, info_ad,iter_ad = solve_sparse(A, b, M=M_ad)